In [9]:
import simulator as sm
import os
file_ = "/home/astro/Documents/ivano_baronchelli_test.fits"
img, hdr = sm.load_fits(file_)
print(img.shape)

obs_date = hdr["DATE-OBS"].split('-')
print(obs_date)


(1, 1, 256, 256)
['2016', '03', '02T00:09:38.448000']


In [11]:
cycles = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
antenna_ids = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

antenna_names = [os.path.join('cycle{}'.format(j), 'alma.cycle{}.0.{}'.format(j, k)) for j, k in zip(cycles, antenna_ids)]
print(antenna_names)

['cycle1/alma.cycle1.0.10', 'cycle2/alma.cycle2.0.9', 'cycle3/alma.cycle3.0.8', 'cycle4/alma.cycle4.0.7', 'cycle5/alma.cycle5.0.6', 'cycle6/alma.cycle6.0.5', 'cycle7/alma.cycle7.0.4', 'cycle8/alma.cycle8.0.3', 'cycle9/alma.cycle9.0.2', 'cycle10/alma.cycle10.0.1']
